Koodi Hamletin enkulle

In [1]:
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')  
nltk.download('omw-1.4') 
nltk.download('stopwords')  

file1 = open("HAMLET.txt", "r")


playEN = [[],
        [],
        [],
        [],
        []]

act = 0

#skips the play information
while True:
    line = file1.readline() 
    if line:
        try:
            if (line[0]+line[1]+line[2]) == "ACT":
                break
            else:
                pass       
        except IndexError:
            pass
            
#get sentences. Contains stage directions
while True:
    line = file1.readline() 
    if line:
        try:
            if (line[0]+line[1]+line[2]) == "ACT":
                act += 1
            if (line[0]+line[1]+line[2]+line[3]+line[4]+line[5]) == "HAMLET":
                if line.split("HAMLET", 1)[1] != "\n":
                    playEN[act].append(line.split("HAMLET", 1)[1].rstrip())
                line = file1.readline()
                while line and line != '\n' :
                    playEN[act].append(line.rstrip())
                    line = file1.readline()
        except IndexError:
            pass
    else :
        file1.close()
        break 

#remove stage direction

noDirectPlayEN = [[x for x in playEN[0] if "[" not in x],
                  [x for x in playEN[1] if "[" not in x],
                  [x for x in playEN[2] if "[" not in x],
                  [x for x in playEN[3] if "[" not in x],
                  [x for x in playEN[4] if "[" not in x]]

#remove special characters
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')

noSpecialCharEN = [[''.join(filter(whitelist.__contains__, x)) for x in noDirectPlayEN[0]],
                   [''.join(filter(whitelist.__contains__, x)) for x in noDirectPlayEN[1]],
                   [''.join(filter(whitelist.__contains__, x)) for x in noDirectPlayEN[2]],
                   [''.join(filter(whitelist.__contains__, x)) for x in noDirectPlayEN[3]],
                   [''.join(filter(whitelist.__contains__, x)) for x in noDirectPlayEN[4]]]

#make everything lower case

allLowerEN = [[ x.lower() for x in noSpecialCharEN[0]],
              [ x.lower() for x in noSpecialCharEN[1]],
              [ x.lower() for x in noSpecialCharEN[2]],
              [ x.lower() for x in noSpecialCharEN[3]],
              [ x.lower() for x in noSpecialCharEN[4]]]

#tokenize

tokensEN = [[ nltk.word_tokenize(x) for x in allLowerEN[0]],
            [ nltk.word_tokenize(x) for x in allLowerEN[1]],
            [ nltk.word_tokenize(x) for x in allLowerEN[2]],
            [ nltk.word_tokenize(x) for x in allLowerEN[3]],
            [ nltk.word_tokenize(x) for x in allLowerEN[4]]]
        
#remove stopwords

stop_wordsEN = set(line.strip() for line in open('stop_words_english.txt'))
noStopwEN = [[[t for t in x if t not in stop_wordsEN ]  for x in tokensEN[0]],
            [ [t for t in x if t not in stop_wordsEN ]  for x in tokensEN[1]],
            [ [t for t in x if t not in stop_wordsEN ]  for x in tokensEN[2]],
            [ [t for t in x if t not in stop_wordsEN ]  for x in tokensEN[3]],
            [ [t for t in x if t not in stop_wordsEN ]  for x in tokensEN[4]]]

#lemmatization
lemmatizer = WordNetLemmatizer()
lemmaEN = [[ [lemmatizer.lemmatize(t) for t in x ]  for x in noStopwEN[0]],
           [ [lemmatizer.lemmatize(t) for t in x ]  for x in noStopwEN[1]],
           [ [lemmatizer.lemmatize(t) for t in x ]  for x in noStopwEN[2]],
           [ [lemmatizer.lemmatize(t) for t in x ]  for x in noStopwEN[3]],
           [ [lemmatizer.lemmatize(t) for t in x ]  for x in noStopwEN[4]]]



#remove act and line seperation
wordsEN = []
for act in lemmaEN:
    for line in act:
        for word in line:
            wordsEN.append(word)

#unique words only
def unique(list1):
  
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    return unique_list

uniqueEN = unique(wordsEN)







[nltk_data] Downloading package wordnet to /home/kali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/kali/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Koodi hamletin ranskalle

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer

file1 = open("HamletFr.txt", "r")


playFR =[[],
         [],
         [],
         [],
         []]

act = 0

#skips the play information
while True:
    line = file1.readline() 
    if line:
        try:
            if (line[0]+line[1]+line[2]+line[3]) == "ACTE":
                break
            else:
                pass       
        except IndexError:
            pass
            
#get sentences
while True:
    line = file1.readline() 
    if line:
        try:
            if (line[0]+line[1]+line[2]+line[3]) == "ACTE":
                act += 1
            if (line[0]+line[1]+line[2]+line[3]+line[4]+line[5]) == "HAMLET":
                if line.split("— ", 1)[1] != "\n":
                    playFR [act].append(line.split("— ", 1)[1].rstrip())
                line = file1.readline()
                while line and line != '\n' :
                    playFR [act].append(line.rstrip())
                    line = file1.readline()
        except IndexError:
            pass
    else :
        file1.close()
        break 


#remove special characters
whitelist = set('abcdefghijklmnopqrstuvwxyzàâèéêëîïôùûüç\'- ÀÂÈÉÊËÎÏÔABCDEFGHIJKLMNOPQRSTUVWXYZ')

noSpecialCharFR  = [[''.join(filter(whitelist.__contains__, x)) for x in playFR[0]],
                    [''.join(filter(whitelist.__contains__, x)) for x in playFR[1]],
                    [''.join(filter(whitelist.__contains__, x)) for x in playFR[2]],
                    [''.join(filter(whitelist.__contains__, x)) for x in playFR[3]],
                    [''.join(filter(whitelist.__contains__, x)) for x in playFR[4]]]

#make everything lower case

allLowerFR  = [[ x.lower() for x in noSpecialCharFR[0]],
               [ x.lower() for x in noSpecialCharFR[1]],
               [ x.lower() for x in noSpecialCharFR[2]],
               [ x.lower() for x in noSpecialCharFR[3]],
               [ x.lower() for x in noSpecialCharFR[4]]]

#tokenize

nlp = spacy.load('fr_core_news_md')
tokenizer = Tokenizer(nlp.vocab)

tokensFR = [[ [t.text for t in tokenizer(x)] for x in allLowerFR[0]],
            [ [t.text for t in tokenizer(x)]  for x in allLowerFR[1]],
            [ [t.text for t in tokenizer(x)]  for x in allLowerFR[2]],
            [ [t.text for t in tokenizer(x)]  for x in allLowerFR[3]],
            [ [t.text for t in tokenizer(x)]  for x in allLowerFR[4]]]
     
#remove stopwords

stop_wordsFR = set(line.strip() for line in open('stop_words_french.txt'))
noStopwFR = [[[t for t in x if t not in stop_wordsFR and t != ' ']  for x in tokensFR[0]],
            [ [t for t in x if t not in stop_wordsFR and t != ' ']  for x in tokensFR[1]],
            [ [t for t in x if t not in stop_wordsFR and t != ' ']  for x in tokensFR[2]],
            [ [t for t in x if t not in stop_wordsFR and t != ' ']  for x in tokensFR[3]],
            [ [t for t in x if t not in stop_wordsFR and t != ' ']  for x in tokensFR[4]]]


#lemmatization. Removes act and line seperation

lemmaFR = []

for act in noStopwFR:
    for line in act:
        for word in line:
            doc = nlp(word)
            for token in doc:
                lemmaFR.append(token.lemma_)


uniqueFR = unique(lemmaFR)








: 

Task 2

In [ ]:
print(len(uniqueEN))
print(len(uniqueFR))

Task 3

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt



averagesEN = []
length = 0

for act in tokensEN:
    for line in act:
        for word in line:
            length += len(word)
        averagesEN.append(round(length / len(line), 2))
        length = 0

averagesFR = []
length = 0

for act in tokensFR:
    for line in act:
        for word in line:
            length += len(word)
        averagesFR.append(round(length / len(line), 2))
        length = 0

fig, ax = plt.subplots()
q25, q75 = np.percentile(averagesEN, [25, 75])
bin_width = 2 * (q75 - q25) * len(averagesEN) ** (-1/3)
bins = round((max(averagesEN)- min(averagesEN)) / bin_width)

sns.histplot(averagesEN, bins=bins, ax=ax, kde=True);
sns.histplot(averagesFR, bins=bins, ax=ax,  kde=True);

Task 4

In [ ]:
import pandas as pd
import math

zipWords = []

frequency = []

#Vaihto tassa kielien valilla
for act in noStopwFR:
    for line in act:
        for word in line:
            zipWords.append(word)

frequency = {}

for word in zipWords:
    count = frequency.get(word , 0)
    frequency[ word ] = count + 1

rank = 1
column_header = ['Rank', 'Frequency', 'Frequency * Rank']
df = pd.DataFrame( columns = column_header )
collection = sorted(frequency.items(), key=itemgetter(1), reverse = True)

for word , freq in collection:
    df.loc[word] = [rank, freq, rank*freq]
    rank = rank + 1

f, ax = plt.subplots(figsize=(10, 7))


sns.regplot(x="Rank", y="Frequency",  data=np.log10(df[:20].astype('float64')), ax=ax)


formatter = lambda x, pos: f'{10 ** x:g}'
ax.get_xaxis().set_major_formatter(formatter)
ax.get_yaxis().set_major_formatter(formatter)

plt.show()